In [1]:
# 세션 유지 코드
from IPython.display import clear_output
import threading, time

def keep_alive():
    for i in range(100000):
        time.sleep(60)
        clear_output(wait=True)
        print(f"Ping {i} ⏱️")

threading.Thread(target=keep_alive).start()

In [ ]:
# ===================================================
# 0. Install & Environment Setup
# ===================================================
!pip install transformers accelerate peft trl bitsandbytes datasets --quiet

import os, gc, re, torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    TrainingArguments, DataCollatorForLanguageModeling
)
from peft import LoraConfig
from trl import SFTTrainer
from huggingface_hub import login

# Optional: Hugging Face login
login(token="YOUR_HF_TOKEN")

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

In [3]:
# ===================================================
# 1. Load MathX-5M and Format Training Text
# ===================================================
print("🔄 Streaming MathX-5M dataset...")
streaming_dataset = load_dataset("XenArcAI/MathX-5M", split="train", streaming=True)
subset_dataset = streaming_dataset.take(30000)

SYSTEM_PROMPT = """You are GemmaMathTutor, a professional math tutor for children. You guide students step-by-step with clear logic and explanations using LaTeX."""

def format_sample(sample):
    question = sample["problem"]
    solution = re.sub(r"<think>.*?</think>", "", sample["generated_solution"], flags=re.DOTALL).strip()
    response = f"""Thinking process: To solve this problem, we need to follow a structured approach, applying the relevant mathematical principles.
Step-by-step calculation:
{solution}
Final Answer: **Answer: (See the final step in the calculation above)**"""
    return {
        "text": (
            f"<start_of_turn>system\n{SYSTEM_PROMPT}<end_of_turn>\n"
            f"<start_of_turn>user\n{question}<end_of_turn>\n"
            f"<start_of_turn>model\n{response}<end_of_turn>"
        )
    }

print("📦 Formatting samples...")
subset_list = list(subset_dataset)
formatted_data = [format_sample(s) for s in subset_list]
train_dataset = Dataset.from_pandas(pd.DataFrame(formatted_data))

🔄 Streaming MathX-5M dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/213 [00:00<?, ?it/s]

📦 Formatting samples...


In [4]:
# ===================================================
# 2. Load Gemma 3n Model (QLoRA) and Tokenizer
# ===================================================
model_id = "google/gemma-3n-E2B-it"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto"
)

Ping 0 ⏱️


In [5]:
# ===================================================
# 3. LoRA Config and Tokenize
# ===================================================
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

def tokenize(sample):
    return tokenizer(sample["text"], padding="max_length", truncation=True, max_length=1024)

tokenized_dataset = train_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [6]:
# ===================================================
# 4. Cast Weights to float32 for Stability
# ===================================================
model.config.altup_coef_clip = 1e-5  # optional clamp

def cast_gemma3n_weights_to_float32(model):
    for name, module in model.named_modules():
        if hasattr(module, "prediction_coefs") and hasattr(module.prediction_coefs, "weight"):
            module.prediction_coefs.weight.data = module.prediction_coefs.weight.data.to(torch.float32)
        if hasattr(module, "correction_coefs") and hasattr(module.correction_coefs, "weight"):
            module.correction_coefs.weight.data = module.correction_coefs.weight.data.to(torch.float32)

cast_gemma3n_weights_to_float32(model)

In [8]:
# ===================================================
# 5. TrainingArguments + SFTTrainer
# ===================================================
training_args = TrainingArguments(
    output_dir="./gemma3n-math-tutor",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=25,
    save_steps=500,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    optim="paged_adamw_8bit",
    report_to=None,
    bf16=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    peft_config=lora_config,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

print("🚀 Starting training...")
##trainer.train()
trainer.train(resume_from_checkpoint="./gemma3n-math-tutor/checkpoint-12500")

Ping 133 ⏱️


Step,Training Loss
12525,5.595600
12550,6.046700
12575,5.808200
12600,5.751200
12625,5.652100
12650,5.349800
12675,5.865200
12700,5.656400
12725,5.840000
12750,5.936600


TrainOutput(global_step=15000, training_loss=0.9413678904215494, metrics={'train_runtime': 7921.4412, 'train_samples_per_second': 7.574, 'train_steps_per_second': 1.894, 'total_flos': 1.06625625882624e+18, 'train_loss': 0.9413678904215494})

In [9]:
# ===================================================
# 6. Save Model + Push to Hugging Face
# ===================================================
save_path = "/content/drive/MyDrive/gemma3n_lora_math"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

repo_id = "LeannaJ/gemma3n-lora-math"
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)
print(f"✅ Model pushed to Hugging Face: https://huggingface.co/{repo_id}")

Ping 137 ⏱️


README.md: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

✅ Model pushed to Hugging Face: https://huggingface.co/LeannaJ/gemma3n-lora-math
